In [1]:

import Thymio as th
import time
import videopath1 as vp
import numpy as np
import threading
import vision.detect_aruco as da
import queue
import cv2
def draw_path(img, path):
    for i in range(len(path)-1):
        cv2.line(img, path[i], path[i+1], (0, 255, 0), 2)
    return img

In [2]:

channel = queue.Queue(maxsize=2)
aruco_thread = threading.Thread(target=da.main,args=(channel,), daemon=True)
aruco_thread.start()
print("Open thymio")
thymio = th.Thymio()
await thymio.initiateLock()
print("Thymio opened")
query = channel.get()



camera_on = query[0]
original_image = query[1]
display_image = query[2]

start = query[3]
print(f"start: {start}")
cv2.imshow("mainframe", display_image)
cv2.waitKey(1)

while not camera_on:
    query = channel.get()
    camera_on = query[0]
    original_image = query[1]
    display_image = query[2]
    start = query[3]
    cv2.imshow("mainframe", display_image)
    cv2.waitKey(1)
    print("waiting for camera", end="\r")

print("first pos queried",start)
path_cm, path, costmap, block_height, block_width, start, goal, display_image, cm_per_pixel =vp.init(original_image,start)
#cv2.imshow("Select Goal", original_image)
#cv2.setMouseCallback("Select Goal", vp.select_points, original_image)
cm_p_pixel = 77/block_width
#path = vp.update(costmap,block_height,block_width,start,vp.points[0],original_image,cm_p_pixel,[])
next_goal = path[0]
current_pos = np.array([start[0],start[1],start[2]])
current_speed = np.array([0,0,0])
camera_position = start
z_predicted = np.concatenate([current_pos,current_speed]) #kalman_state
sigma_predicted = np.diag([0.1,0.1,0.1,0,0,0])



vars_towait = ['prox.horizontal','motor.right.speed','motor.left.speed','motor.right.target','motor.right.target']
vars_thread = threading.Thread(target=thymio.wait_for_variables, args=(vars_towait,))
vars_thread.start()
time.sleep(1)

while True:
    query = channel.get()
    camera_on = query[0]
    #original_image = query[1]
    display_image = query[2]
    start = query[3]
    
    #draw_path(display_image, path_cm)
    cv2.imshow("going there", display_image)
    cv2.waitKey(1)
    print(path, end="\r")
    next_goal = path[0]
    print(f'Next goal : {next_goal}')
    print(f'Current position : {current_pos}')
    thymio.navigate(current_pos, next_goal)
    current_pos = z_predicted[:3] # Is used for the controller
    speedright = thymio.getSpeedR()
    speedleft = thymio.getSpeedL()
    
    z_filtered, sigma_filtered= thymio.filtering_step(z_k_k_1 = z_predicted, 
                                    sigma_k_k_1 = sigma_predicted, 
                                    V_left_measure =speedleft, 
                                    V_right_measure = speedright, 
                                    camera_working=camera_on, 
                                    x_measured=camera_position[0], 
                                    y_measured=camera_position[1],
                                    theta_measured= camera_position[2])
    
    z_predicted, sigma_predicted = thymio.prediction_step(z_filtered, sigma_filtered)

    obstacle,detected = thymio.getObstaclePosition()
    if detected:
        print("Obstacle detected")
        #TODO: use the obstacle position to update the costmap
        original_image = query[1]
        path,costmap = vp.update(costmap,block_height,block_width,current_pos,vp.points[0],original_image,cm_p_pixel,obstacle)
        
    if thymio.robot_close_waypoint(current_pos[:2], next_goal[0], next_goal[1]):
        path.pop(0)
        if len(path) == 0:
            print("Goal reached")
            thymio.set_multiple_variables({"motor.left.target":[0],"motor.right.target":[0]})
            vars_thread.join()
            thymio.stop()
            aruco_thread.join()
            break
        
        

[INFO] detecting 'DICT_ARUCO_ORIGINAL' markers...Open thymio

Thymio opened
2
4
3
5
1
start: (86.54750705550329, 896.4646464646464, 1.5707963267948966)
4
3
5
1
2
first pos queried (86.54750705550329, 896.4646464646464, 1.5707963267948966)
2
4
3
5
1
3
2
4
5
1


TypeError: unsupported operand type(s) for /: 'tuple' and 'int'

3
2
4
5
1
2
4
3
5
1
3
2
4
5
1
3
2
4
5
1
2
4
3
5
1
3
2
4
5
1
2
4
3
5
1
3
2
4
5
1
3
2
4
5
1
3
2
4
5
1
2
4
3
5
1
3
2
4
5
1
2
4
3
5
1
3
2
4
5
1
3
2
4
5
1
2
4
3
5
1
2
4
3
5
1
3
2
4
5
1
2
4
3
5
1
2
4
3
5
1
3
1
2
4
5
3
2
4
5
1
3
2
4
5
1
3
2
4
5
1
3
2
4
5
1
2
4
3
5
1
3
2
4
5
1
3
2
4
5
1
2
4
3
5
1
2
4
3
5
1
2
4
3
5
1
3
2
4
5
1
2
4
3
5
1
3
2
4
5
1
3
2
4
5
1
3
2
4
5
1
2
4
3
5
1
3
2
4
5
1
3
5
2
4
1
2
4
3
5
1
3
2
4
5
1
2
4
3
5
1
3
2
4
5
1
3
2
4
5
1
3
2
4
5
1
3
2
4
5
1
3
2
4
5
1
2
4
3
5
1
3
2
4
5
1
2
4
3
5
1
3
2
4
5
1
3
2
4
5
1
3
2
4
5
1
3
2
4
5
1
3
2
4
5
1
2
4
3
5
1
3
2
4
5
1
3
2
4
5
1
2
4
3
5
1
3
2
4
5
1
2
4
3
5
1
3
2
4
5
1
3
2
4
5
1
3
2
4
5
1
3
2
4
5
1
3
2
4
5
1
3
2
4
5
1
3
2
4
5
1
3
2
4
5
1
3
2
4
5
1
3
2
4
5
1
3
2
4
5
1
3
2
4
5
1
2
4
3
5
1
2
4
3
5
1
3
2
4
5
1
3
2
4
5
1
3
2
4
5
1
3
2
4
5
1
3
5
2
4
1
3
2
4
5
1
2
4
3
5
1
3
2
4
5
1
3
2
4
5
1
3
2
4
5
1
3
2
4
5
1
2
4
3
5
1
3
2
4
5
1
3
2
4
5
1
3
2
4
5
1
3
2
4
5
1
3
2
4
5
1
2
4
3
5
1
3
2
4
5
1
2
4
3
5
1
2
4
3
5
1
2
4
3
5
1
3
2
4
5
1
3
2
4
5
1
3
2
4
5
1


In [2]:
import cv2
print(cv2.__version__)

4.6.0
